# Clean Data

## chapter 1 Why Do You Need Clean Data

连接本地数据库中Enron的邮件数据库。

dataset name: enron

In [17]:
import pandas as pd
import pymysql

conn = pymysql.connect(host="localhost", user='root', password='1q2w3e4r5t', db='enron')
sql = "select * from message"
enron_msgs = pd.read_sql(sql, con=conn)

In [3]:
enron_msgs.head()

,mid,sender,date,message_id,subject,body,folder
0,52,press.release@enron.com,2000-01-21 04:51:00,<12435833.1075863606729.JavaMail.evans@thyme>,ENRON HOSTS ANNUAL ANALYST CONFERENCE PROVIDES...,HOUSTON - Enron Corp. hosted its annual equity...,Robert_Badeer_Aug2000Notes FoldersPress releases
1,53,office.chairman@enron.com,2000-01-24 01:37:00,<29664079.1075863606676.JavaMail.evans@thyme>,Over $50 -- You made it happen!,"On Wall Street, people are talking about Enron...",Robert_Badeer_Aug2000Notes FoldersPress releases
2,54,office.chairman@enron.com,2000-01-24 02:06:00,<15300605.1075863606629.JavaMail.evans@thyme>,Over $50 -- You made it happen!,"On Wall Street, people are talking about Enron...",Robert_Badeer_Aug2000Notes FoldersPress releases
3,55,press.release@enron.com,2000-02-02 10:21:00,<10522232.1075863606538.JavaMail.evans@thyme>,ROAD-SHOW.COM Q4i.COM CHOOSE ENRON TO DELIVER ...,"HOUSTON =01) Enron Broadband Services (EBS), a...",Robert_Badeer_Aug2000Notes FoldersPress releases
4,56,office.chairman@enron.com,2000-02-07 22:16:00,<16056123.1075863606515.JavaMail.evans@thyme>,Fortune Most Admired Ranking,Congratulations! For an unprecedented five ye...,Robert_Badeer_Aug2000Notes FoldersPress releases


In [4]:
# 用SQL语句，统计每天的记录数量，并生成dataframe数据
sql_date = 'select date(date) as dateSent, count(mid) as numMsg from message GROUP BY dateSent ORDER BY dateSent'
group_msgs = pd.read_sql(sql_date, con=conn)
group_msgs.head()

,dateSent,numMsg
0,0001-05-30,2
1,0001-06-18,1
2,0001-07-24,1
3,0001-08-01,1
4,0001-08-06,1


In [6]:
group_msgs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1323 entries, 0 to 1322
Data columns (total 2 columns):
dateSent    1323 non-null object
numMsg      1323 non-null int64
dtypes: int64(1), object(1)
memory usage: 20.8+ KB


用pandas的函数，对In[17]中生成的enron_msgs进行分组统计，得到与In[4]同样的结果。


In [7]:
enron_msgs.dtypes

mid            int64
sender        object
date          object
message_id    object
subject       object
body          object
folder        object
dtype: object

In [27]:
enron_msgs['date'].min()

datetime.datetime(1, 5, 30, 13, 10, 6)

In [28]:
enron_msgs['date'].max()

datetime.datetime(2044, 1, 4, 14, 48, 58)

In [29]:
# 将date特征转化为日期型，因为存在上述的错误数据，所以，以下参数中errors = 'coerce'
temp = pd.to_datetime(enron_msgs['date'], format="%Y-%m-%d", errors = 'coerce')
temp.head()

0   2000-01-21 04:51:00
1   2000-01-24 01:37:00
2   2000-01-24 02:06:00
3   2000-02-02 10:21:00
4   2000-02-07 22:16:00
Name: date, dtype: datetime64[ns]

In [31]:
enron_msgs['justday'] = temp.dt.date

In [44]:
import datetime
group_msgs[group_msgs['dateSent']==datetime.datetime.strptime('1979-12-31', '%Y-%m-%d').date()]

,dateSent,numMsg
103,1979-12-31,6


In [46]:
group_msgs2 = enron_msgs.groupby('justday').count()
group_msgs2.head()

,mid,sender,date,message_id,subject,body,folder
justday,,,,,,,
1979-12-31,6,6,6,6,6,6,6
1997-01-01,1,1,1,1,1,1,1
1998-01-04,1,1,1,1,1,1,1
1998-01-05,1,1,1,1,1,1,1
1998-10-30,3,3,3,3,3,3,3


对照In[4]和In[46]两个分组统计结果，在In[4]中，存在不符合正常时期或者日期书写不规范的结果，In[46]中，则都是按照”年、月、日“方式书写的数据，其他不符合规范的，已经在In[29]中转化为NaN.

In [48]:
sql2 = 'SELECT date(date) AS dateSent, count(mid) AS numMsg From message WHERE year(date) BETWEEN 1998 AND 2002 GROUP BY dateSent ORDER BY datesent;'
group_msgs3 = pd.read_sql(sql2, con=conn)
group_msgs3.head()

,dateSent,numMsg
0,1998-01-04,1
1,1998-01-05,1
2,1998-10-30,3
3,1998-11-02,1
4,1998-11-03,1
